In [1]:
import os
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm, LinearSegmentedColormap

# sklearn, general tools
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, \
    roc_auc_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay, \
    precision_recall_curve, roc_curve, plot_roc_curve

# sklearn, models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

---

# open data

In [ ]:
# misc settings

classlabel = {  # class labels and their names
    "k-2pi" : 0,
    "k-pinunu" : 1,  # signal
    "lambda-pin" : 5,
    "background" : 99,  # all the channels but k-pinunu
}

bTossFaultyVertex = True  # toss faulty vertex data?

limLambdaWeights = [1e-8, 100]  # range limits to the lambda weights; set lower to 0 to keep all

bOnlyInFV = False  # limit the whole analysis to the FV?
shift_fv = 150  # shift between absolute and relative (in data) position of the FV
fv_edges = (280, 350)  # FV boundaries, in absolute longitudinal coordinate
bOnlyInPtCut = False  # limit the whole analysis to within the cut on the pion p_t?
pt_cut = 0.140  # lower cut on the pion p_t

bBinaryClass = True  # if True (False), only two classes chosen below (all the classes) are kept
whichsig = "k-pinunu"  # class to be used as signal? (set regardless of value of bBinaryClass)
whichbkg = "lambda-pin"  # class to be used as background? (if bBinaryClass is False, overwritten to "background")
nEvsPop = 100000  # (max.) nr. of events per class - if needed: set None to skip
nMultLambda = 1  # nMultLambda*nEvsPop lambda or tot. backgroud background events will be loaded (at max.)

bReweightStat = False  # reweight each class according to expected statistics? (if False, normalise each class to 1)
nb = 5*200*3000  # nr. of bursts in the whole experiment lifetime
ppb = 2e13  # nr. of protons on target per burst
kpp = 2.1e-5  # nr. of kaons per proton on target
beamcorr = (0.256/0.4)**2  # ratio between beam solid angles for short and long tunnel
lambdacorr = (37/73)*(2.00/2.19)  # correction to the kaon production yield for lambda
classflux = {  # expected flux of parent particles out of the target
    "k-2pi" : nb*ppb*kpp*beamcorr,
    "k-pinunu" : nb*ppb*kpp*beamcorr,
    "lambda-pin" : nb*ppb*kpp*beamcorr*lambdacorr,
}
classbr = {  # BR of each decay class
    "k-2pi" : 0.864e-3,
    "k-pinunu" : 3.0e-11,
    "lambda-pin" : 0.358,
}
#classflux = {  # expected flux of parent particles out of the target
#    "k-2pi" : 0.446e12,
#    "k-pinunu" : 15483,
#    "lambda-pin" : 8.22e13,
#}
#classbr = {  # BR of each decay class
#    "k-2pi" : 1,
#    "k-pinunu" : 1,
#    "lambda-pin" : 1,
#}

classngen = {  # zOptical initial events per class
    "k-2pi" : 7e9,
    "k-pinunu" : 70e6,
    "lambda-pin" : 100e6,
}

split_test_fraction = 0.5  # fraction of total statistics to be used for testing

confusionmatrix_norm = "pred"  # set normalisation rule for the confusion matrices (true/pred/all)

In [ ]:
# data folder(s) (shall have "/" at the end):
masterpath = "/DATA_MASTER_PATH/"
datapath = [
    masterpath + "23_hike_pinunu-background/2305_zoptical-zanalyze_training/",
    #masterpath + "23_hike_pinunu-background/2306_zoptical-zanalyze_lambda_mass_prod/",
    #masterpath + "23_hike_pinunu-background/2306_zoptical-zanalyze_2pi_mass_prod/",
]

# data selection criteria, set here:
datasel = lambda name : ("2305_zoptical-zanalyze_training" in name) & (".csv" in name) & ("_V2_" in name)
#def datasel(name):
#    sel = ("2305_zoptical-zanalyze_training" in name) & (".csv" in name) & ("_V2_" in name)
#    for i in range(100):  # this limits the dataset to a fixed amount of mass-production files
#        if i<11:
#            sel = sel | (
#                ("2306_zoptical-zanalyze_lambda_mass_prod" in name) &\
#                ("040623" in name) &\
#                ("_"+str(i)+"." in name)
#            )
#    return sel

filenames = []
for s in datapath:
    filenames += [s+f.name for f in list(os.scandir(s)) if datasel(s+f.name)]
print("list of files to open:")
print(*filenames, sep="\n")

In [ ]:
%%time

# open data
df = pd.DataFrame()
for name in filenames:
    print("opening file %s" % name.rsplit('/', 1)[-1])
    df0 = pd.read_csv(name)
    df0["file"] = name.replace(".csv", "")
    #df = pd.concat([df, df0], ignore_index=True, sort=False)
    df = pd.concat([df, df0[  # quick trick to deal with lambda high (useless) statistics
        ((df0["class"]==classlabel["lambda-pin"]) & (df0.W>limLambdaWeights[0]) & (df0.W<limLambdaWeights[1])) |\
        (df0["class"]!=classlabel["lambda-pin"])
    ]], ignore_index=True, sort=False)
    del df0
    
# duplicate raw weights
df["W0"] = df.W
    
# there are faulty vertex (as reconstructed by the calorimeter) data --> throwing them away if needed
if bTossFaultyVertex:
    print("removing %d broken-vertex events (%f%%)" % (
        df[df.Vertex_xRec_Z.isnull()].shape[0], df[df.Vertex_xRec_Z.isnull()].shape[0]/df.shape[0]*100
    ))
    df = df[~df.Vertex_xRec_Z.isnull()]
    
# crude nr. of events
print("---")
print("total events: %d" % df.shape[0])
for i_class in df["class"].unique():
    s_class=[s for s in classlabel if classlabel[s]==i_class][0]
    print("events in class %d (%s): %d" % (
        i_class, s_class, df[df["class"]==i_class].shape[0])
    )

# sum of event weights
sumWOrig = {}
for i_class in df["class"].unique():
    s_class=[s for s in classlabel if classlabel[s]==i_class][0]
    print(
        "sum of weights in class %s: %f" % (
            s_class, sum(df[df["class"]==classlabel[s_class]].W0)
        )
    )
    sumWOrig.update({i_class: sum(df[df["class"]==classlabel[s_class]].W0)})
    
print("---")

In [ ]:
# cut on lambda weights

if limLambdaWeights[0]>0:
    s_class = "lambda-pin"
    i_class = classlabel[s_class]
    
    df = df[
        ((df["class"]==i_class) & (df.W0>limLambdaWeights[0]) & (df.W0<limLambdaWeights[1])) |\
        (df["class"]!=i_class)
    ]
    
    print("after selecting only (lambda) weights > %.2E & < %.2E:" % (limLambdaWeights[0], limLambdaWeights[1]))
    print("events in class %d (%s): %d" % (
        i_class, s_class, df[df["class"]==i_class].shape[0])
    )
    print(
        "sum of weights in class %s: %f" % (
            s_class, sum(df[df["class"]==i_class].W0)
        )
    )
    
else:
    print("keeping the whole lambda distribution")

In [ ]:
# only keep events in the blind analysis signal box, if needed

fv = (fv_edges[0]-shift_fv, fv_edges[1]-shift_fv)

if bOnlyInFV | bOnlyInPtCut:
    x_box = df.Vertex_xRec_Z
    y_box = np.sqrt(
        (df.Vertex_pRec0_X + df.Vertex_pRec1_X)**2 + (df.Vertex_pRec0_Y + df.Vertex_pRec1_Y)**2
    )
    
    bool_fv = ((x_box > fv[0]) & (x_box < fv[1])) \
        if bOnlyInFV else (x_box < 1e10)
    bool_pt = y_box > (pt_cut if bOnlyInPtCut else -1)
    
    df = df[bool_fv & bool_pt]
    
    print("after limiting to the blind analysis box:")
    
    # crude nr. of events
    print("total events: %d" % df.shape[0])
    for i_class in df["class"].unique():
        s_class=[s for s in classlabel if classlabel[s]==i_class][0]
        print("events in class %d (%s): %d" % (
            i_class, s_class, df[df["class"]==i_class].shape[0])
        )
            
    # sum of event weights
    sumWOrig = {}  # if limiting to the FV and/or the cut in pt, reset sumWOrig
    for i_class in df["class"].unique():
        s_class=[s for s in classlabel if classlabel[s]==i_class][0]
        print(
            "sum of weights in class %s: %f" % (
                s_class, sum(df[df["class"]==classlabel[s_class]].W0)
            )
        )
        sumWOrig.update({i_class: sum(df[df["class"]==classlabel[s_class]].W0)})
        
else:
    print("keeping the whole final-variable phase space")

In [ ]:
# descale to a fixed dataset size (per class)

if not (nEvsPop is None):  # deactivate setting nEvsPop=None above
    print("after descaling (w/ %d events per class - *%d in case of lambda):" % (nEvsPop, nMultLambda))

    classdf = []
    for i, i_class in enumerate(df["class"].unique()):
        if bBinaryClass:
            classdf.append(
                df[df["class"]==i_class].sample(frac=1).head(
                    nEvsPop \
                    if (i_class!=classlabel["lambda-pin"]) else \
                    nMultLambda*nEvsPop
                ))
        else:
            classdf.append(
                df[df["class"]==i_class].sample(frac=1).head(
                    nEvsPop \
                    if (i_class==classlabel[whichsig]) else \
                    int(nEvsPop / (len(df["class"].unique())-1))
                ))
        print("events in class %d: %d" % (i_class, classdf[i].shape[0]))

    df = pd.concat(classdf)
    print("total events: %d" % df.shape[0])
    
else:
    print("no descaling is done")

In [ ]:
# fix the event weights - part 1/2 (must be done before manipulating classes)

for i_class in df["class"].unique():
    s_class = [s for s in classlabel.keys() if classlabel[s]==i_class][0]

    df.loc[df["class"]==i_class, "W2"] = df.W0 * (
        (classbr[s_class]*classflux[s_class]) / \
        (classngen[s_class] * (sum(df[df["class"]==classlabel[s_class]].W0)/sumWOrig[classlabel[s_class]]))
        # num. is the nr. of expected decays in the experiment (overall, not only the properly detected ones)
        # den. is the nr. of events initially simulated in zOptical (rescaled to match the nEvsPop subsel.)
    )
    print(
        "class %s, sum of weights (after descaling and rescaling - W2): %f" % (
            s_class, sum(df[df["class"]==classlabel[s_class]].W2)
        )
    )

In [ ]:
# check all available classes

for i_class in df["class"].unique():
    print(
        "class = %d <--> filename type is e.g. %s" % (
            i_class, str(df[df["class"] == i_class].file.head(1).values[0]).rsplit('/', 1)[-1]
        )
    )
    
df["class0"] = df["class"]

if bBinaryClass:
    df = df[(df["class"]==classlabel[whichsig]) | (df["class"]==classlabel[whichbkg])]
    print("only keeping %s (signal) and %s (background)" % (whichsig, whichbkg))
else:
    df.loc[df["class"]!=classlabel[whichsig], "class"] = classlabel["background"]
    whichbkg = "background"
    print(
        "keeping all the available channels (%d) but turning all but signal into tot. background (%d)" \
        % (len(df["class"].unique()), classlabel["background"])
    )

In [ ]:
# fix the event weights - part 2/2 (must be done after manipulating classes)

for i_class in df["class"].unique():
    s_class = [s for s in classlabel.keys() if classlabel[s]==i_class][0]
    
    df.loc[df["class"]==i_class, "W1"] = df.W0 / sum(df[df["class"]==i_class].W0)
    print(
        "class %s, sum of weights (after descaling and normalising - W1): %f" % (
            s_class, sum(df[df["class"]==classlabel[s_class]].W1)
        )
    )
    
df["W"] = (df.W2) if bReweightStat else (df.W1)
s = "eventually using weights "+("normalised to 1" if (not bReweightStat) else "rescaled to the exp. stat.")+" per class"
s += (" (W1)" if (not bReweightStat) else " (W2)")
print(s)

In [ ]:
print("list of variables (%d):" % df.shape[1])
print(df.columns)

---

# create classification datasets

creating

- `y`, series with class indexex
- `W`, series with weights
- `X`, dataframe with only the variables selected for classification

### CLASSES AND WEIGHTS

In [ ]:
# class index - signal will be 1, background will be 0
y0 = label_binarize(df["class"].copy(), classes=[classlabel[whichbkg], classlabel[whichsig]], neg_label=0, pos_label=1)
y0 = np.ravel(y0)
y = pd.Series(data=y0, index=df.index)

In [ ]:
# event weights
W = df["W"].copy()  # can be physical weights or normalised weights, depending on bReweightStat
WPhys = df["W2"].copy()  # is always the physical weights

### VARIABLES

In [ ]:
%%time

# classification variables (only the "actually measured" quantities & all those derived from them)

# select best vertex quantities
df.loc[df.Vertex_nConverted==0, "Vertex_xBest_Z"] = df.Vertex_xRec_Z
df.loc[df.Vertex_nConverted!=0, "Vertex_xBest_Z"] = df.Vertex_xRecPre_Z
df.loc[df.Vertex_nConverted==0, "Vertex_xBest_X"] = df.Vertex_xRec_X
df.loc[df.Vertex_nConverted!=0, "Vertex_xBest_X"] = df.Vertex_xRecPre_X
df.loc[df.Vertex_nConverted==0, "Vertex_xBest_Y"] = df.Vertex_xRec_Y
df.loc[df.Vertex_nConverted!=0, "Vertex_xBest_Y"] = df.Vertex_xRecPre_Y
df.loc[df.Vertex_nConverted==0, "Vertex_pBest0_Z"] = df.Vertex_pRec0_Z
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest0_Z"] = df.Vertex_pRecPre0_Z
df.loc[df.Vertex_nConverted==0, "Vertex_pBest0_X"] = df.Vertex_pRec0_X
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest0_X"] = df.Vertex_pRecPre0_X
df.loc[df.Vertex_nConverted==0, "Vertex_pBest0_Y"] = df.Vertex_pRec0_Y
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest0_Y"] = df.Vertex_pRecPre0_Y
df.loc[df.Vertex_nConverted==0, "Vertex_pBest1_Z"] = df.Vertex_pRec1_Z
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest1_Z"] = df.Vertex_pRecPre1_Z
df.loc[df.Vertex_nConverted==0, "Vertex_pBest1_X"] = df.Vertex_pRec1_X
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest1_X"] = df.Vertex_pRecPre1_X
df.loc[df.Vertex_nConverted==0, "Vertex_pBest1_Y"] = df.Vertex_pRec1_Y
df.loc[df.Vertex_nConverted!=0, "Vertex_pBest1_Y"] = df.Vertex_pRecPre1_Y

# compute all transverse quantities from cartesian components
df["Vertex_xRec_T"] = np.sqrt( df.Vertex_xRec_X**2 + df.Vertex_xRec_Y**2 )
df["Vertex_xRecPre_T"] = np.sqrt( df.Vertex_xRecPre_X**2 + df.Vertex_xRecPre_Y**2 )
df["Cluster0_xRec_T"] = np.sqrt( df.Cluster0_xRec_X**2 + df.Cluster0_xRec_Y**2 )
df["Cluster1_xRec_T"] = np.sqrt( df.Cluster1_xRec_X**2 + df.Cluster1_xRec_Y**2 )
df["Vertex_pRec0_T"] = np.sqrt( df.Vertex_pRec0_X**2 + df.Vertex_pRec0_Y**2 )
df["Vertex_pRec1_T"] = np.sqrt( df.Vertex_pRec1_X**2 + df.Vertex_pRec1_Y**2 )
df["Cluster0_xPre1_T"] = np.sqrt( df.Cluster0_xPre1_X**2 + df.Cluster0_xPre1_Y**2 )
df["Cluster0_xPre2_T"] = np.sqrt( df.Cluster0_xPre2_X**2 + df.Cluster0_xPre2_Y**2 )
df["Cluster1_xPre1_T"] = np.sqrt( df.Cluster1_xPre1_X**2 + df.Cluster1_xPre1_Y**2 )
df["Cluster1_xPre2_T"] = np.sqrt( df.Cluster1_xPre2_X**2 + df.Cluster1_xPre2_Y**2 )
df["Vertex_pRecPre0_T"] = np.sqrt( df.Vertex_pRecPre0_X**2 + df.Vertex_pRecPre0_Y**2 )
df["Vertex_pRecPre1_T"] = np.sqrt( df.Vertex_pRecPre1_X**2 + df.Vertex_pRecPre1_Y**2 )

df["Vertex_xBest_T"] = np.sqrt( df.Vertex_xBest_X**2 + df.Vertex_xBest_Y**2 )
df["Vertex_pBest0_T"] = np.sqrt( df.Vertex_pBest0_X**2 + df.Vertex_pBest0_Y**2 )
df["Vertex_pBest1_T"] = np.sqrt( df.Vertex_pBest1_X**2 + df.Vertex_pBest1_Y**2 )

# compute new 2-cluster position and momentum-related variables
df["Clusters_xRec_TMin"] = df[["Cluster0_xRec_T", "Cluster1_xRec_T"]].min(axis=1)
df["Clusters_xPre1_TMin"] = df[["Cluster0_xPre1_T", "Cluster1_xPre1_T"]].min(axis=1)
df["Clusters_xPre2_TMin"] = df[["Cluster0_xPre2_T", "Cluster1_xPre2_T"]].min(axis=1)
df["Vertex_pRec_TMin"] = df[["Vertex_pRec0_T", "Vertex_pRec1_T"]].min(axis=1)
df["Vertex_pRecPre_TMin"] = df[["Vertex_pRecPre0_T", "Vertex_pRecPre1_T"]].min(axis=1)
df["Clusters_xRec_TMax"] = df[["Cluster0_xRec_T", "Cluster1_xRec_T"]].max(axis=1)
df["Clusters_xPre1_TMax"] = df[["Cluster0_xPre1_T", "Cluster1_xPre1_T"]].max(axis=1)
df["Clusters_xPre2_TMax"] = df[["Cluster0_xPre2_T", "Cluster1_xPre2_T"]].max(axis=1)
df["Vertex_pRec_TMax"] = df[["Vertex_pRec0_T", "Vertex_pRec1_T"]].max(axis=1)
df["Vertex_pRecPre_TMax"] = df[["Vertex_pRecPre0_T", "Vertex_pRecPre1_T"]].max(axis=1)
df["Clusters_xRec_TSum"] = df.Clusters_xRec_TMax + df.Clusters_xRec_TMin
df["Clusters_xPre1_TSum"] = df.Clusters_xPre1_TMax + df.Clusters_xPre1_TMin
df["Clusters_xPre2_TSum"] = df.Clusters_xPre2_TMax + df.Clusters_xPre2_TMin
df["Vertex_pRec_TSum"] = df.Vertex_pRec_TMax + df.Vertex_pRec_TMin
df["Vertex_pRecPre_TSum"] = df.Vertex_pRecPre_TMax + df.Vertex_pRecPre_TMin
df["Clusters_xRec_TDif"] = df.Clusters_xRec_TMax - df.Clusters_xRec_TMin
df["Clusters_xPre1_TDif"] = df.Clusters_xPre1_TMax - df.Clusters_xPre1_TMin
df["Clusters_xPre2_TDif"] = df.Clusters_xPre2_TMax - df.Clusters_xPre2_TMin
df["Vertex_pRec_TDif"] = df.Vertex_pRec_TMax - df.Vertex_pRec_TMin
df["Vertex_pRecPre_TDif"] = df.Vertex_pRecPre_TMax - df.Vertex_pRecPre_TMin
df["Clusters_xRec_TAsym"] = df.Clusters_xRec_TDif / df.Clusters_xRec_TSum
df["Clusters_xPre1_TAsym"] = df.Clusters_xPre1_TDif / df.Clusters_xPre1_TSum
df["Clusters_xPre2_TAsym"] = df.Clusters_xPre2_TDif / df.Clusters_xPre2_TSum
df["Vertex_pRec_TAsym"] = df.Vertex_pRec_TDif / df.Vertex_pRec_TSum
df["Vertex_pRecPre_TAsym"] = df.Vertex_pRecPre_TDif / df.Vertex_pRecPre_TSum

df["Vertex_pBest_TMin"] = df[["Vertex_pBest0_T", "Vertex_pBest1_T"]].min(axis=1)
df["Vertex_pBest_TMax"] = df[["Vertex_pBest0_T", "Vertex_pBest1_T"]].max(axis=1)
df["Vertex_pBest_TSum"] = df.Vertex_pBest_TMax + df.Vertex_pBest_TMin
df["Vertex_pBest_TDif"] = df.Vertex_pBest_TMax - df.Vertex_pBest_TMin
df["Vertex_pBest_TAsym"] = df.Vertex_pBest_TDif / df.Vertex_pBest_TSum

df["Clusters_xDist"] = np.sqrt(  # distance between clusters
    (df.Cluster1_xRec_X-df.Cluster0_xRec_X)**2 + (df.Cluster1_xRec_Y-df.Cluster0_xRec_Y)**2
)

# compute new 2-cluster energy-related variables
df["Clusters_EMin"] = df[["Cluster0_ERec", "Cluster1_ERec"]].min(axis=1)
df["Clusters_EMax"] = df[["Cluster0_ERec", "Cluster1_ERec"]].max(axis=1)
df["Clusters_ESum"] = df.Clusters_EMax + df.Clusters_EMin
df["Clusters_EDif"] = df.Clusters_EMax - df.Clusters_EMin
df["Clusters_EAsym"] = df.Clusters_EDif / df.Clusters_ESum

df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Clusters_xRec_TEMin"] = df.Cluster0_xRec_T
df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Clusters_xRec_TEMax"] = df.Cluster1_xRec_T
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Clusters_xRec_TEMin"] = df.Cluster1_xRec_T
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Clusters_xRec_TEMax"] = df.Cluster0_xRec_T

df["Clusters_ECOG"] = np.sqrt(  # cog. between clusters (weighted with energies)
    (
        (df.Cluster0_ERec*df.Cluster0_xRec_X + df.Cluster1_ERec*df.Cluster1_xRec_X)**2 +\
        (df.Cluster0_ERec*df.Cluster0_xRec_Y + df.Cluster1_ERec*df.Cluster1_xRec_Y)**2
    ) / df.Clusters_ESum**2
)

# compute pion kinematics from two photons (pion energy is simply Clusters_ESum)
df["Vertex_pRecPi_Z"] = df.Vertex_pRec0_Z + df.Vertex_pRec1_Z
df["Vertex_pRecPi_X"] = df.Vertex_pRec0_X + df.Vertex_pRec1_X
df["Vertex_pRecPi_Y"] = df.Vertex_pRec0_Y + df.Vertex_pRec1_Y
df["Vertex_pRecPrePi_Z"] = df.Vertex_pRecPre0_Z + df.Vertex_pRecPre1_Z
df["Vertex_pRecPrePi_X"] = df.Vertex_pRecPre0_X + df.Vertex_pRecPre1_X
df["Vertex_pRecPrePi_Y"] = df.Vertex_pRecPre0_Y + df.Vertex_pRecPre1_Y

df["Vertex_pRecPi_T"] = np.sqrt( df.Vertex_pRecPi_X**2 + df.Vertex_pRecPi_Y**2 )
df["Vertex_pRecPrePi_T"] = np.sqrt( df.Vertex_pRecPrePi_X**2 + df.Vertex_pRecPrePi_Y**2 )

df["Vertex_pRecPi_sTh"] = df.Vertex_pRecPi_T / np.sqrt(  # pion propagation angle wrt. beam
    df.Vertex_pRecPi_T**2 + df.Vertex_pRecPi_Z**2
)
df["Vertex_pRecPrePi_sTh"] = df.Vertex_pRecPrePi_T / np.sqrt(  # pion propagation angle wrt. beam
    df.Vertex_pRecPrePi_T**2 + df.Vertex_pRecPrePi_Z**2
)

df["Vertex_pBestPi_Z"] = df.Vertex_pBest0_Z + df.Vertex_pBest1_Z
df["Vertex_pBestPi_X"] = df.Vertex_pBest0_X + df.Vertex_pBest1_X
df["Vertex_pBestPi_Y"] = df.Vertex_pBest0_Y + df.Vertex_pBest1_Y

df["Vertex_pBestPi_T"] = np.sqrt( df.Vertex_pBestPi_X**2 + df.Vertex_pBestPi_Y**2 )

df["Vertex_pBestPi_sTh"] = df.Vertex_pBestPi_T / np.sqrt(  # pion propagation angle wrt. beam
    df.Vertex_pBestPi_T**2 + df.Vertex_pBestPi_Z**2
)

# new ideas
df.loc[df.Vertex_nConverted==0, "Vertex_x_ZDiffPreRec"] = 9999
df.loc[df.Vertex_nConverted!=0, "Vertex_x_ZDiffPreRec"] = df.Vertex_xRec_Z - df.Vertex_xRecPre_Z 
# ^^^ Matt: diff. between vertex position from MEC only and from MEC+PSD, longitudinal

df.loc[df.Vertex_nConverted==0, "Vertex_x_TDiffPreRec"] = 9999
df.loc[df.Vertex_nConverted!=0, "Vertex_x_TDiffPreRec"] = df.Vertex_xRec_T - df.Vertex_xRecPre_T  
# ^^^ Matt: diff. between vertex position from MEC only and from MEC+PSD, transverse

df.loc[df.Vertex_nConverted==0, "Vertex_x_DistDiffPreRec"] = 9999
df.loc[df.Vertex_nConverted!=0, "Vertex_x_DistDiffPreRec"] = np.sqrt(  
    (df.Vertex_xRec_X - df.Vertex_xRecPre_X)**2 +\
    (df.Vertex_xRec_Y - df.Vertex_xRecPre_Y)**2 +\
    (df.Vertex_xRec_Z - df.Vertex_xRecPre_Z)**2
)  # Matt: diff. between vertex position from MEC only and from MEC+PSD, total

df["Clusters_ERatio"] = df.Clusters_EMin / df.Clusters_EMax  # from KOTO - lowest-to-highest energy ratio

df["Vertex_pRec0_sTh"] = df.Vertex_pRec0_T / np.sqrt(
    df.Vertex_pRec0_T**2 + df.Vertex_pRec0_Z**2
)
df["Vertex_pRec1_sTh"] = df.Vertex_pRec1_T / np.sqrt(
    df.Vertex_pRec1_T**2 + df.Vertex_pRec1_Z**2
)
df["Vertex_pRecPre0_sTh"] = df.Vertex_pRecPre0_T / np.sqrt(
    df.Vertex_pRecPre0_T**2 + df.Vertex_pRecPre0_Z**2
)
df["Vertex_pRecPre1_sTh"] = df.Vertex_pRecPre1_T / np.sqrt(
    df.Vertex_pRecPre1_T**2 + df.Vertex_pRecPre1_Z**2
)
df["Vertex_Rec0_EsTh"] = df.Cluster0_ERec*df.Vertex_pRec0_sTh 
df["Vertex_Rec1_EsTh"] = df.Cluster1_ERec*df.Vertex_pRec1_sTh
df["Vertex_RecPre0_EsTh"] = df.Cluster0_ERec*df.Vertex_pRecPre0_sTh
df["Vertex_RecPre1_EsTh"] = df.Cluster1_ERec*df.Vertex_pRecPre1_sTh
# ^^^ KOTO - single-photon angle*energy 

df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Vertex_Rec_EMinsTh"] = df.Vertex_Rec0_EsTh
df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Vertex_Rec_EMaxsTh"] = df.Vertex_Rec1_EsTh
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Vertex_Rec_EMinsTh"] = df.Vertex_Rec1_EsTh
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Vertex_Rec_EMaxsTh"] = df.Vertex_Rec0_EsTh
df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Vertex_RecPre_EMinsTh"] = df.Vertex_RecPre0_EsTh
df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Vertex_RecPre_EMaxsTh"] = df.Vertex_RecPre1_EsTh
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Vertex_RecPre_EMinsTh"] = df.Vertex_RecPre1_EsTh
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Vertex_RecPre_EMaxsTh"] = df.Vertex_RecPre0_EsTh
# ^^^ KOTO - single-photon angle*energy 

df["Vertex_pBest0_sTh"] = df.Vertex_pBest0_T / np.sqrt(
    df.Vertex_pBest0_T**2 + df.Vertex_pBest0_Z**2
)
df["Vertex_pBest1_sTh"] = df.Vertex_pBest1_T / np.sqrt(
    df.Vertex_pBest1_T**2 + df.Vertex_pBest1_Z**2
)
df["Vertex_Best0_EsTh"] = df.Cluster0_ERec*df.Vertex_pBest0_sTh
df["Vertex_Best1_EsTh"] = df.Cluster1_ERec*df.Vertex_pBest1_sTh
# ^^^ KOTO - single-photon angle*energy 

df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Vertex_Best_EMinsTh"] = df.Vertex_Best0_EsTh
df.loc[df.Cluster0_ERec==df.Clusters_EMin, "Vertex_Best_EMaxsTh"] = df.Vertex_Best1_EsTh
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Vertex_Best_EMinsTh"] = df.Vertex_Best1_EsTh
df.loc[df.Cluster1_ERec==df.Clusters_EMin, "Vertex_Best_EMaxsTh"] = df.Vertex_Best0_EsTh
# ^^^ KOTO - single-photon angle*energy 

df["Vertex_pRec0_sPhi"] = np.arcsin(df.Vertex_pRec0_Y / df.Vertex_pRec0_T)
df["Vertex_pRec1_sPhi"] = np.arcsin(df.Vertex_pRec1_Y / df.Vertex_pRec1_T)
df["Vertex_pRecPre0_sPhi"] = np.arcsin(df.Vertex_pRecPre0_Y / df.Vertex_pRecPre0_T)
df["Vertex_pRecPre1_sPhi"] = np.arcsin(df.Vertex_pRecPre1_Y / df.Vertex_pRecPre1_T)
df["Vertex_pRec_dsPhi"] = np.abs(df.Vertex_pRec1_sPhi - df.Vertex_pRec0_sPhi)
df["Vertex_pRecPre_dsPhi"] = np.abs(df.Vertex_pRecPre1_sPhi - df.Vertex_pRecPre0_sPhi)
# ^^^ KOTO - transv. angle between photons
    
df["Vertex_pBest0_sPhi"] = np.arcsin(df.Vertex_pBest0_Y / df.Vertex_pBest0_T)
df["Vertex_pBest1_sPhi"] = np.arcsin(df.Vertex_pBest1_Y / df.Vertex_pBest1_T)
df["Vertex_pBest_dsPhi"] = np.abs(df.Vertex_pBest1_sPhi - df.Vertex_pBest0_sPhi)
# ^^^ KOTO - transv. angle between photons

print("(preliminary) nr. of classification variables after calculations: %d" % df.shape[1])
print("---")

In [ ]:
# final variable selection

X = df[[
    
    # variables for final analysis
    #"Vertex_xBest_Z",
    #"Vertex_pBestPi_T",

    # vertex data (computed either with MEC only or with MEC & PSD)
    "Vertex_xBest_T",

    # cluster data - positions on MEC
    #"Clusters_xRec_TMin",  # highly correlated with Clusters_xDist
    #"Clusters_xRec_TMax",  # highly correlated with Clusters_xDist
    "Clusters_xRec_TAsym",
    "Clusters_xDist",
    "Clusters_ECOG",

    # cluster data - energy
    "Clusters_EMin",
    #"Clusters_EMax",  # highly correlated with Clusters_ESum
    "Clusters_ESum",
    #"Clusters_EAsym",  # highly correlated with Clusters_ERatio
    
    # new ideas
    "Vertex_x_ZDiffPreRec",
    #"Vertex_x_TDiffPreRec",
    #"Vertex_x_DistDiffPreRec",
    "Clusters_ERatio",
    "Vertex_Best_EMinsTh",
    "Vertex_Best_EMaxsTh",
    "Vertex_pBest_dsPhi",
]]

### CHECK QUALITY OF SELECTED VARIABLES

In [ ]:
# violin plot, dataset creation

bPlotViolin = False  # create and plot it?

if bPlotViolin:
    
    # copy of X with all variables normalised to 1
    X_norm = X.copy()
    for s in X.columns:
        X_norm[s] = (X_norm[s] - X_norm[s].mean()) / X_norm[s].std()
    
    # dedicated dataframe for violin plot
    X_viol = pd.melt(pd.concat([X_norm, y], axis=1), "class", var_name="feature")
    for s in classlabel:
        X_viol.loc[X_viol["class"]==classlabel[s], "class"] = s

In [ ]:
%%time

# violin plot, plotting (if bPlotViolin is True and class choice is compatible)

if bPlotViolin:
    if (bBinaryClass | (not (classlabel["lambda-pin"] in df["class"].unique()))):
        bPlotViolin_save = True  # save plot?
        scale = "width"  # scale heights to the bin entries (count) or to the distr. maxima (width)

        plt.figure(figsize=(12, 7))
        plt.ylim((-3, 6))
        plt.yscale("linear")

        viol = sns.violinplot(
            x="feature", y="value", hue="class",
            split=True, data=X_viol, inner=None, bw=0.05, gridsize=1000, scale=scale
        )
        viol.set_xticklabels(rotation=90, labels=list(X_norm)) ;

        plt.grid()
        plt.tight_layout()
        if bPlotViolin_save:
            plt.savefig("./output_misc/violinplot.png")
            
    else:
        print("cannot make a class-comparison violin plot with this class conf.")

In [ ]:
# correlation matrix

bCorrMatrix = True  # plot it?

if bCorrMatrix:
    
    bCorrMatrix_save = True  # save plot?

    plt.figure(figsize=(12, 8))
    sns.heatmap(X.corr(), linewidths=.1, annot=True, annot_kws={"size": 10}, cmap="PiYG", center=0)
    plt.yticks(rotation=0) ;
    
    plt.tight_layout()
    if bCorrMatrix_save:
        plt.savefig("./output_misc/correlationmatrix.png")

### MORE CHECKS, MOSTLY FOR LAMBDA BACKGROUND

In [ ]:
# min, mean, median, max for each variable

if False:
    for var in X.columns:
        print(f"{var:>23}{X[var].min():^28}{X[var].mean():^23}{X[var].median():^23}{X[var].max():^23}")

In [ ]:
# check all variables with standard histograms -- as a function of the longitudinal position

perc_fv = 0.85

if False:
    for var in X.columns:
        fig, ax = plt.subplots(ncols=2, nrows=3, figsize=(12, 10), sharex=True)
        ax[0, 0].grid(True)
        ax[0, 1].grid(True)
        ax[1, 0].grid(True)
        ax[1, 1].grid(True)
        ax[2, 0].grid(True)
        ax[2, 1].grid(True)
        ax[0, 0].set_title("all events")
        ax[0, 1].set_title("no conversions in PSD")
        ax[1, 0].set_title("1 conversion in PSD")
        ax[1, 1].set_title("2 conversions in PSD")
        ax[2, 0].set_title("nonzero, upstream of %d%% of the FV" % (perc_fv*100))
        ax[2, 1].set_title("nonzero, downsteam of %d%% of the FV" % (100-perc_fv*100))
        
        bins_sp={  # manual tweak to the binning below (l. edge, r. edge, bins for non-lambda, bins for lambda)
            "Vertex_xBest_T" : (0, 0.3, 120, 60),
            "Clusters_EMin" : (0, 45, 120, 60),
            "Clusters_ESum" : (0, 100, 120, 60),
            "Vertex_x_DistDiffPreRec" : (0, 300, 120, 60),
            "Vertex_x_ZDiffPreRec" : (-100, 300, 120, 60),
            "Vertex_x_TDiffPreRec" : (-0.2, 0.1, 120, 60),
            "Vertex_Best_EMinsTh" : (0, 0.5, 120, 60),
            "Vertex_Best_EMaxsTh" : (0, 1, 120, 60),
        }
        bins = (
            np.linspace(
                min(df[df["class"]!=classlabel["lambda-pin"]][var]) if (not (var in bins_sp)) else bins_sp[var][0],
                max(df[df["class"]!=classlabel["lambda-pin"]][var]) if (not (var in bins_sp)) else bins_sp[var][1],
                120 if (not (var in bins_sp)) else bins_sp[var][2]  # binning for all the classes but lambda
            ),
            np.linspace(
                min(df[df["class"]!=classlabel["lambda-pin"]][var]) if (not (var in bins_sp)) else bins_sp[var][0],
                max(df[df["class"]!=classlabel["lambda-pin"]][var]) if (not (var in bins_sp)) else bins_sp[var][1],
                60 if (not (var in bins_sp)) else bins_sp[var][3]  # binning for lambda
            ),
        )
        
        for i_class in df["class"].unique():
            b = (df["class"]==i_class)

            ax[0, 0].hist(
                df[b][var], 
                bins=bins[0] if i_class!=classlabel["lambda-pin"] else bins[1], histtype="step", density=True,
                label=[s for s in classlabel if classlabel[s]==i_class][0], 
                weights=df[b].W
            )

            ax[0, 1].hist(
                df[b & (df.Vertex_nConverted==0)][var], 
                bins=bins[0] if i_class!=classlabel["lambda-pin"] else bins[1], histtype="step", density=True,
                label=[s for s in classlabel if classlabel[s]==i_class][0], 
                weights=df[b & (df.Vertex_nConverted==0)].W
            )

            ax[1, 0].hist(
                df[b & (df.Vertex_nConverted==1)][var], 
                bins=bins[0] if i_class!=classlabel["lambda-pin"] else bins[1], histtype="step", density=True,
                label=[s for s in classlabel if classlabel[s]==i_class][0], 
                weights=df[b & (df.Vertex_nConverted==1)].W
            )

            ax[1, 1].hist(
                df[b & (df.Vertex_nConverted==2)][var], 
                bins=bins[0] if i_class!=classlabel["lambda-pin"] else bins[1], histtype="step", density=True,
                label=[s for s in classlabel if classlabel[s]==i_class][0], 
                weights=df[b & (df.Vertex_nConverted==2)].W
            )

            ax[2, 0].hist(
                df[b & (df[var]!=0) & (df.Vertex_xRec_Z<(fv[0]+perc_fv*(fv[1]-fv[0])))][var], 
                bins=bins[0] if i_class!=classlabel["lambda-pin"] else bins[1], histtype="step", density=True,
                label=[s for s in classlabel if classlabel[s]==i_class][0], 
                weights=df[b & (df[var]!=0) & (df.Vertex_xRec_Z<(fv[0]+perc_fv*(fv[1]-fv[0])))].W
            )

            ax[2, 1].hist(
                df[b & (df[var]!=0) & (df.Vertex_xRec_Z>(fv[0]+perc_fv*(fv[1]-fv[0])))][var], 
                bins=bins[0] if i_class!=classlabel["lambda-pin"] else bins[1], histtype="step", density=True,
                label=[s for s in classlabel if classlabel[s]==i_class][0], 
                weights=df[b & (df[var]!=0) & (df.Vertex_xRec_Z>(fv[0]+perc_fv*(fv[1]-fv[0])))].W
            )

        ax[0, 0].legend()
        fig.supxlabel(var)

        fig.tight_layout()
        if True:
            plt.savefig("./output_misc/check_vars_%s.png" % var)

---

# BDT hyperparameter settings

In [ ]:
# create training/testing datasets for the new model
X_train, X_test, y_train, y_test, W_train, W_test, WPhys_train, WPhys_test =\
    train_test_split(X, y, W, WPhys, test_size=split_test_fraction)
print("total classification dataset size is %d" % y.shape[0])
print("training (testing) size is %d (%d)" % (y_train.shape[0], y_test.shape[0]))

In [ ]:
# custom scoring functions

hyperparams_opt = {}  # this needs to be equalised even in case the optimisation isn't run

## specific scorer for HIKE, binary problem - outdated
#def custom_singlebkg_score(
#    y_true, y_pred, *,
#    whichsig, whichbkg,
#    classngen, classlabel, classbr, classflux,
#    sample_weight=None, reweight=False, sample_weight_phys=None,
#):  
#    
#    # number of events initially simulated in zOptical (rescaled according to the nEvsPop subselection)
#    nact_S = classngen[whichsig] * (len(y_true[y_true==classlabel[whichsig]]) / sumWOrig[classlabel[whichsig]])
#    nact_B = classngen[whichbkg] * (len(y_true[y_true==classlabel[whichbkg]]) / sumWOrig[classlabel[whichbkg]])
#    
#    # number of expected decays in the experiment (overall, not only the properly detected ones)
#    nexp_S = classbr[whichsig] * classflux[whichsig]
#    nexp_B = classbr[whichbkg] * classflux[whichbkg]
#    
#    if sample_weight is None:
#    # if no weights are provided at all, simply count all the events classified in each class --> bad
#        Ss = np.count_nonzeroes((y_pred==classlabel[whichsig]) & (y_true==classlabel[whichsig])) * (nexp_S/nact_S)
#        Bs = np.count_nonzeroes((y_pred==classlabel[whichsig]) & (y_true==classlabel[whichbkg])) * (nexp_B/nact_B)  # TBC
#        
#    else:
#        sample_weight = sample_weight.loc[y_true.index.values].values.reshape(-1)
#        
#        if not reweight:
#        # with normalised weights, get the sum of physical (i.e. rescaled) event weights in each class
#            if sample_weight_phys==None:  # if no physical weights are given, use the normalised ones
#                sample_weight_phys = sample_weight
#            Ss = sum(sample_weight_phys[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichsig])])
#            Bs = sum(sample_weight_phys[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichbkg])])
#            
#        else:
#        # with physical (i.e. rescaled) weights, simply get the weight sum of events in each class
#            Ss = sum(sample_weight[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichsig])])
#            Bs = sum(sample_weight[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichbkg])])
#    
#    return Ss / np.sqrt(Bs + Ss)

## specific scorer for HIKE, binary problem - outdated
#def custom_singlebkg_score(
#    y_true, y_pred, *,
#    whichsig, whichbkg, classlabel
#    sample_weight=None, reweight=False, sample_weight_phys=None,
#):  
#    
#    sample_weight = sample_weight.loc[y_true.index.values].values.reshape(-1)
#
#    if not reweight:
#    # with normalised weights, get the sum of physical (i.e. rescaled) event weights in each class, if available
#    # (in the scorer only, normalised weights used everywhere else)
#        if sample_weight_phys==None:  # if no physical weights are given, use the normalised ones
#            sample_weight_phys = sample_weight
#        Ss = sum(sample_weight_phys[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichsig])])
#        Bs = sum(sample_weight_phys[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichbkg])])
#
#    else:
#    # with physical (i.e. rescaled) weights, simply get the weight sum of events in each class
#        Ss = sum(sample_weight[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichsig])])
#        Bs = sum(sample_weight[(y_pred==classlabel[whichsig]) & (y_true==classlabel[whichbkg])])
#    
#    return Ss / np.sqrt(Bs + Ss)

# specific scorer for HIKE, binary problem
def custom_singlebkg_hike_score(
    y_true, y_pred, *,
    sample_weight=None, reweight=False, sample_weight_phys=None,
):  
    
    if not (sample_weight is None):
        sample_weight = sample_weight.loc[y_true.index.values].values.reshape(-1)
    if not (sample_weight_phys is None):
        sample_weight_phys = sample_weight_phys.loc[y_true.index.values].values.reshape(-1)
        
    # compute optimal cut and use it to separate predicted signal and background
    sig_cst = []
    p_min = min(y_pred)
    p_max = max(y_pred)
    p_cuts = np.linspace(p_min, p_max, 200)
    for p_cut in p_cuts:
        y_pred_temp = (y_pred > p_cut).astype('float')
        sig_sel = np.count_nonzero(y_pred_temp*y_true)
        bkg_sel = np.count_nonzero(y_pred_temp*(1-y_true))
        sig_rej = np.count_nonzero((1-y_pred_temp)*y_true)
        bkg_rej = np.count_nonzero((1-y_pred_temp)*(1-y_true))
        sig_cst.append(
            (sig_sel / (sig_sel+bkg_sel)**0.5) if (sig_sel+bkg_sel)!=0 else 0
        )
    opt_cut = p_cuts[5:-5][np.array(sig_cst[5:-5])==max(sig_cst)][0]
    y_pred_int = (y_pred > opt_cut).astype('float')
    print("--> hike, obtained optimal cut is %f" % opt_cut)

    if not reweight:
    # with normalised weights, get the sum of physical (i.e. rescaled) event weights in each class, if available
    # (in the scorer only, normalised weights used everywhere else)
        if (sample_weight_phys is None):  # if no physical weights are given, use the normalised ones
            sample_weight_phys = sample_weight
        Ss = sum(sample_weight_phys[(y_pred_int==1) & (y_true==1)])
        Bs = sum(sample_weight_phys[(y_pred_int==1) & (y_true==0)])

    else:
    # with physical (i.e. rescaled) weights, simply get the weight sum of events in each class
        Ss = sum(sample_weight[(y_pred_int==1) & (y_true==1)])
        Bs = sum(sample_weight[(y_pred_int==1) & (y_true==0)])
    
    return Ss / np.sqrt(Bs + Ss)

# confusion matrix (with optimised cut) determinant, binary problem
def custom_singlebkg_confmatrdet_score(
    y_true, y_pred, *,
    sample_weight=None,
):  
    
    if not (sample_weight is None):
        sample_weight = sample_weight.loc[y_true.index.values].values.reshape(-1)
    
    # compute optimal cut and use it to separate predicted signal and background
    sig_cst = []
    p_min = min(y_pred)
    p_max = max(y_pred)
    p_cuts = np.linspace(p_min, p_max, 200)
    for p_cut in p_cuts:
        y_pred_temp = (y_pred > p_cut).astype('float')
        sig_sel = np.count_nonzero(y_pred_temp*y_true)
        bkg_sel = np.count_nonzero(y_pred_temp*(1-y_true))
        sig_rej = np.count_nonzero((1-y_pred_temp)*y_true)
        bkg_rej = np.count_nonzero((1-y_pred_temp)*(1-y_true))
        sig_cst.append(
            (sig_sel / (sig_sel+bkg_sel)**0.5) if (sig_sel+bkg_sel)!=0 else 0
        )
    opt_cut = p_cuts[5:-5][np.array(sig_cst[5:-5])==max(sig_cst)][0]
    y_pred_int = (y_pred > opt_cut).astype('float')
    print("--> confmatrdet, obtained optimal cut is %f" % opt_cut)
    
    confusionmatrix = confusion_matrix(y_true, y_pred_int, sample_weight=sample_weight, normalize=confusionmatrix_norm)
    
    return np.linalg.det(confusionmatrix)

# balanced accuracy computed with the optimised cut, binary problem
def custom_singlebkg_balancedaccuracy_score(
    y_true, y_pred, *,
    sample_weight=None,
):  
    
    if not (sample_weight is None):
        sample_weight = sample_weight.loc[y_true.index.values].values.reshape(-1)
    
    # compute optimal cut and use it to separate predicted signal and background
    sig_cst = []
    p_min = min(y_pred)
    p_max = max(y_pred)
    p_cuts = np.linspace(p_min, p_max, 200)
    for p_cut in p_cuts:
        y_pred_temp = (y_pred > p_cut).astype('float')
        sig_sel = np.count_nonzero(y_pred_temp*y_true)
        bkg_sel = np.count_nonzero(y_pred_temp*(1-y_true))
        sig_rej = np.count_nonzero((1-y_pred_temp)*y_true)
        bkg_rej = np.count_nonzero((1-y_pred_temp)*(1-y_true))
        sig_cst.append(
            (sig_sel / (sig_sel+bkg_sel)**0.5) if (sig_sel+bkg_sel)!=0 else 0
        )
    opt_cut = p_cuts[5:-5][np.array(sig_cst[5:-5])==max(sig_cst)][0]
    y_pred_int = (y_pred > opt_cut).astype('float')
    print("--> balancedaccuracy, obtained optimal cut is %f" % opt_cut)
    
    # normalise weights
    norm_weight = np.zeros(len(sample_weight))
    norm_weight[y_true==0] = sample_weight[y_true==0]/sum(sample_weight[y_true==0])
    norm_weight[y_true==1] = sample_weight[y_true==1]/sum(sample_weight[y_true==1])
    
    Ss = sum(norm_weight[(y_pred_int==1) & (y_true==1)])
    Br = sum(norm_weight[(y_pred_int==0) & (y_true==0)])
    
    return (1/sum(norm_weight))*(Ss+Br)

In [ ]:
%%time

# classifier optimised with cross-validated scan over hyperparameter grid

bClsGscv = True  # perform this analysis? if False, use previously selected hyperparameters
bPerfPlots_save = True  # save performance plots?
k_fold_index = 5  # nr. of folds to create from the total training set for k-folding

classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())  # algorithm to use
hyperparams = {  # list of hyperparameters to scan over
    "base_estimator__max_depth" : [2, 5, 10, 20],  # max_depth - maximum branch depth, set 1 for stumps
    "n_estimators" : [200, 500, 1000, 2000],  # n_trees - number of trees
    "learning_rate" : [1],  # alpha - AdaBoost learning rate
    "base_estimator__min_weight_fraction_leaf" : [0.05],  # minimum leaf size allowed (stops splitting after that)
}

scorers = {  # list of scoring estimators
    "custom_singlebkg_balancedaccuracy_score" : make_scorer(
        custom_singlebkg_balancedaccuracy_score, sample_weight=W,
        greater_is_better=True, needs_threshold=False, needs_proba=True,
    ),
    "custom_singlebkg_hike_score" : make_scorer(
        custom_singlebkg_hike_score,
        sample_weight=W, reweight=bReweightStat, sample_weight_phys=WPhys,
        greater_is_better=True, needs_threshold=False, needs_proba=True,
    ),
    "custom_singlebkg_confmatrdet_score" : make_scorer(
        custom_singlebkg_confmatrdet_score,
        sample_weight=W,
        greater_is_better=True, needs_threshold=False, needs_proba=True,
    ),
}
scorer_opt = "custom_singlebkg_hike_score"  # choose scoring estimator wrt. which optimise the classifier

if bClsGscv:
    
    # initialise the k-folding splitter
    skf = StratifiedKFold(n_splits=k_fold_index)
    print(
        "%d-folding will exploit training folds of %d events each" \
        % (k_fold_index, y_train.shape[0]/k_fold_index)
    )
    
    print("---")
    
    # initialise & fit the scan --> find the best tree
    cls_gscv = GridSearchCV(
        classifier, refit=scorer_opt,
        param_grid=hyperparams, scoring=scorers, cv=skf, return_train_score=True, verbose=4,
    )
    cls_gscv.fit(X_train, y_train, sample_weight=W_train)
    hyperparams_opt = cls_gscv.best_params_
    print("---\nbest-classifier hyperparameters for %s:" % scorer_opt)
    print(hyperparams_opt)
    
    print("---")

In [ ]:
if bClsGscv:  # set above

    # scoring comparison between different hyperparameter combinations
    if True:
    
        # set the 2 hyperparameters (xplot, yplot) and the scoring estimators (zplot - list) to consider
        xplot = "param_base_estimator__max_depth"
        yplot = "param_n_estimators"
        zplot = [
            "custom_singlebkg_balancedaccuracy_score", 
            "custom_singlebkg_confmatrdet_score",
            "custom_singlebkg_hike_score",
        ]

        dt_scores = pd.DataFrame(cls_gscv.cv_results_)
        print("available variables:")
        print(dt_scores.columns)

        fig, ax = plt.subplots(figsize=(12, 8), ncols=len(zplot), nrows=2)
        for i, s in enumerate(zplot):
            sns.heatmap(
                dt_scores[[xplot, yplot, "mean_train_"+s]].pivot(
                    index=xplot, columns=yplot, values="mean_train_"+s
                ), 
                ax=ax[0, i], linewidths=.1, edgecolors="k", annot=True, 
                annot_kws={"size": 9}, fmt=".3", cmap="viridis"
            )
            sns.heatmap(
                dt_scores[[xplot, yplot, "mean_test_"+s]].pivot(
                    index=xplot, columns=yplot, values="mean_test_"+s
                ), 
                ax=ax[1, i], linewidths=.1, edgecolors="k", annot=True, 
                annot_kws={"size": 9}, fmt=".3", cmap="viridis"
            )
            ax[0, i].set_title("mean_train, "+s.replace("custom_", "").replace("_score", ""))
            ax[1, i].set_title("mean_test, "+s.replace("custom_", "").replace("_score", ""))
        fig.tight_layout()

        if bPerfPlots_save:
            fig.savefig("./output_misc/cls_gscv_grids.png")

---

# final BDT

also saving the trained classifier to `./output_misc/bdt_ab.pickle`

In [ ]:
# create training/testing datasets for the new model
X_train, X_test, y_train, y_test, W_train, W_test, WPhys_train, WPhys_test =\
    train_test_split(X, y, W, WPhys, test_size=split_test_fraction)
print("total classification dataset size is %d" % y.shape[0])
print("training (testing) size is %d (%d)" % (y_train.shape[0], y_test.shape[0]))

In [ ]:
%%time

# single BDT with AdaBoost with selected hyperparameters

bBdtAb = True  # do it?
bPerfPlots_save = True  # save confusion matrix plots?
bClassifier_save = True  # save the trained model?
bManualSet = True  # set parameters manually or take those from optimisation above, if available?
max_depth = \
    5 if (bManualSet | (len(hyperparams_opt)<=1)) \
    else hyperparams_opt["base_estimator__max_depth"]  # maximum branch depth, set 1 for stumps
n_trees = \
    200 if (bManualSet | (len(hyperparams_opt)<=1)) \
    else hyperparams_opt["n_estimators"]  # number of trees
alpha = \
    1 if (bManualSet | (len(hyperparams_opt)<=1)) \
    else hyperparams_opt["learning_rate"]  # AdaBoost learning rate
min_weight_fraction_leaf = \
    0.05 if (bManualSet | (len(hyperparams_opt)<=1)) \
    else hyperparams_opt["base_estimator__min_weight_fraction_leaf"]  # minimum leaf size allowed (stops splitting after that)
output_cut_man = 0.5  # manual set of the optimised decision cut
bManualCut = False  # if False, output_cut_man is overridden by an automatic cut below
bAutoCutTrain = True  # if True (False), compute automatic cut from training (testing) - useless if bManualCut is False

if bBdtAb:
    
    # print chosen parameters
    whichparams = "manually defined" if bManualSet else "grid-borne"
    print("final BDT created with the %s parameters:" % whichparams)
    print("max_depth = %d" % max_depth)
    print("n_trees = %d" % n_trees)
    print("alpha = %f" % alpha)
    print("min_weight_fraction_leaf = %f" % min_weight_fraction_leaf)

    # initialise & fit the classifier
    bdt_ab = AdaBoostClassifier(
        DecisionTreeClassifier(
            max_depth=max_depth,
            min_weight_fraction_leaf=min_weight_fraction_leaf,
        ), 
        n_estimators=n_trees,
        learning_rate=alpha,
    )
    bdt_ab.fit(X_train, y_train, sample_weight=W_train)
    
    # compute the class probability for each event
    y_scores_train_float = bdt_ab.predict_proba(X_train)[:, np.where(bdt_ab.classes_==1)[0][0]]
    y_scores_test_float = bdt_ab.predict_proba(X_test)[:, np.where(bdt_ab.classes_==1)[0][0]]

In [ ]:
if bBdtAb:  # set above
    
    # performance plots, with cut at 50%
    y_scores_train_int = (y_scores_train_float > 0.5).astype('float')
    y_scores_test_int = (y_scores_test_float > 0.5).astype('float')
    
    if True:  # (float) classifier output distributions
        bins = 100  # set nr. of bins for each distribution
        
        fig, ax = plt.subplots(figsize=(12, 5), ncols=2)
        fig.suptitle(
            "%d trees with max. depth %d, learning rate %d, min. tot. weight per leaf %.3f; output cut %f" \
            % (n_trees, max_depth, alpha, min_weight_fraction_leaf, 0.5)
        )
        fig.supxlabel("classifier output")
        ax[0].set_title("training")
        ax[1].set_title("testing")
        
        for i, i_class in enumerate(df["class"].unique()):
            ax[0].hist(
                y_scores_train_float[df["class"][y_train.index]==i_class], bins=bins,
                histtype="step", label="true "+[s for s in classlabel if classlabel[s]==i_class][0]
            )
            ax[1].hist(
                y_scores_test_float[df["class"][y_test.index]==i_class], bins=bins,
                histtype="step", label="true "+[s for s in classlabel if classlabel[s]==i_class][0]
            )
        ax[0].axvline(0.5, ls="--", color="k")
        ax[1].axvline(0.5, ls="--", color="k")
        
        ax[0].grid()
        ax[1].grid()
        ax[1].legend()
        
        fig.tight_layout()
        if bPerfPlots_save:
            fig.savefig("./output_misc/bdt_ab_output.png")
    
    if True:  # confusion matrix
        fig, ax = plt.subplots(figsize=(12, 5), ncols=2)
        fig.suptitle(
            "%d trees with max. depth %d, learning rate %d, min. tot. weight per leaf %.3f; output cut %f" \
            % (n_trees, max_depth, alpha, min_weight_fraction_leaf, 0.5)
        )
        
        ConfusionMatrixDisplay(
            confusion_matrix=confusion_matrix(y_train, y_scores_train_int, normalize=confusionmatrix_norm),
            display_labels=[whichbkg, whichsig],
        ).plot(ax=ax[0])
        ConfusionMatrixDisplay(
            confusion_matrix=confusion_matrix(y_test, y_scores_test_int, normalize=confusionmatrix_norm),
            display_labels=[whichbkg, whichsig],
        ).plot(ax=ax[1])
        
        ax[0].set_title("training (%d events; norm. rule: %s)" % (y_train.shape[0], confusionmatrix_norm))
        ax[1].set_title("testing (%d events; norm. rule: %s)" % (y_test.shape[0], confusionmatrix_norm))
        
        fig.tight_layout()
        if bPerfPlots_save:
            fig.savefig("./output_misc/bdt_ab_confusionmatrix.png")

In [ ]:
if bBdtAb:  # set above
    
    # performance plots, at various thresholds
    
    # do all the necessary calculations anyway
    p_min = min(min(y_scores_train_float), min(y_scores_test_float))
    p_max = max(max(y_scores_train_float), max(y_scores_test_float))
    p_cuts = np.linspace(p_min, p_max, bins)
    
    sig_eff_train = []
    bkg_eff_train = []
    sig_pur_train = []
    sig_cst_train = []
    
    sig_eff_test = []
    bkg_eff_test = []
    sig_pur_test = []
    sig_cst_test = []
    
    for p_cut in p_cuts:
        
        y_scores_train_temp = (y_scores_train_float > p_cut).astype('float')
        sig_sel_train = np.count_nonzero(y_scores_train_temp*y_train)
        bkg_sel_train = np.count_nonzero(y_scores_train_temp*(1-y_train))
        sig_rej_train = np.count_nonzero((1-y_scores_train_temp)*y_train)
        bkg_rej_train = np.count_nonzero((1-y_scores_train_temp)*(1-y_train))
        sig_eff_train.append(
            (sig_sel_train / (sig_sel_train+sig_rej_train)) if (sig_sel_train+sig_rej_train)!=0 else 0
        )
        bkg_eff_train.append(
            (bkg_sel_train / (bkg_sel_train+bkg_rej_train)) if (bkg_sel_train+bkg_rej_train)!=0 else 0
        )
        sig_pur_train.append(
            (sig_sel_train / (sig_sel_train+bkg_sel_train)) if (sig_sel_train+bkg_sel_train)!=0 else 0
        )
        sig_cst_train.append(
            (sig_sel_train / (sig_sel_train+bkg_sel_train)**0.5) if (sig_sel_train+bkg_sel_train)!=0 else 0
        )
        
        y_scores_test_temp = (y_scores_test_float > p_cut).astype('float')
        sig_sel_test = np.count_nonzero(y_scores_test_temp*y_test)
        bkg_sel_test = np.count_nonzero(y_scores_test_temp*(1-y_test))
        sig_rej_test = np.count_nonzero((1-y_scores_test_temp)*y_test)
        bkg_rej_test = np.count_nonzero((1-y_scores_test_temp)*(1-y_test))
        sig_eff_test.append(
            (sig_sel_test / (sig_sel_test+sig_rej_test)) if (sig_sel_test+sig_rej_test)!=0 else 0
        )
        bkg_eff_test.append(
            (bkg_sel_test / (bkg_sel_test+bkg_rej_test)) if (bkg_sel_test+bkg_rej_test)!=0 else 0
        )
        sig_pur_test.append(
            (sig_sel_test / (sig_sel_test+bkg_sel_test)) if (sig_sel_test+bkg_sel_test)!=0 else 0
        )
        sig_cst_test.append(
            (sig_sel_test / (sig_sel_test+bkg_sel_test)**0.5) if (sig_sel_test+bkg_sel_test)!=0 else 0
        )
        
    opt_cut_train = p_cuts[5:-5][np.array(sig_cst_train[5:-5])==max(sig_cst_train)][0] if\
                    len(p_cuts[5:-5][np.array(sig_cst_train[5:-5])==max(sig_cst_train)])==1 else\
                    np.mean(p_cuts[5:-5][np.array(sig_cst_train[5:-5])==max(sig_cst_train)])
    opt_cut_test = p_cuts[5:-5][np.array(sig_cst_test[5:-5])==max(sig_cst_test)][0] if\
                    len(p_cuts[5:-5][np.array(sig_cst_test[5:-5])==max(sig_cst_test)])==1 else\
                    np.mean(p_cuts[5:-5][np.array(sig_cst_test[5:-5])==max(sig_cst_test)])
    
    output_cut = output_cut_man if bManualCut else (opt_cut_train if bAutoCutTrain else opt_cut_test)
            
    if True:  # ROC curve
        fig, ax = plt.subplots(figsize=(6, 5))
        fig.suptitle(
            "%d trees with max. depth %d, learning rate %d,\nmin. tot. weight per leaf %.3f" \
            % (n_trees, max_depth, alpha, min_weight_fraction_leaf)
        )
        
        plot_roc_curve(
            bdt_ab, X_train, y_train, sample_weight=W[W_train.index], 
            ax=ax, name="training", pos_label=1
        ) ;
        plot_roc_curve(
            bdt_ab, X_test, y_test, sample_weight=W[W_test.index], 
            ax=ax, name="testing", pos_label=1
        ) ;
        ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", color="k")
        
        ax.grid()
        
        fig.tight_layout()
        if bPerfPlots_save:
            fig.savefig("./output_misc/bdt_ab_roccurve.png")
            
    if True:  # performance scorers
        bins = 200  # set nr. of cuts to probe
        
        fig, ax = plt.subplots(figsize=(12, 5), ncols=2)
        fig.suptitle(
            "%d trees with max. depth %d, learning rate %d, min. tot. weight per leaf %.3f" \
            % (n_trees, max_depth, alpha, min_weight_fraction_leaf)
        )
        ax[0].set_title("training")
        ax[1].set_title("testing")
            
        ax[0].plot(p_cuts[5:-5], sig_eff_train[5:-5], color="C0", label="Ss/(Ss+Sr) i.e. SE")
        ax[0].plot(p_cuts[5:-5], bkg_eff_train[5:-5], color="C1", label="Bs/(Bs+Br)")
        ax[0].plot(p_cuts[5:-5], sig_pur_train[5:-5], color="deepskyblue", ls="--", label="Ss/(Ss+Bs) i.e. SP")
        ax[0].plot(
            p_cuts[5:-5], np.array(sig_eff_train[5:-5])*np.array(sig_pur_train[5:-5]), 
            color="darkblue", ls=":", lw=2, label="SE*SP"
        )
        ax[0].plot(
            p_cuts[5:-5], np.array(sig_cst_train[5:-5])/max(sig_cst_train), color="C2", label="Ss/(Ss+Bs)^0.5 (norm.)"
        )
        ax[1].plot(p_cuts[5:-5], sig_eff_test[5:-5], color="C0", label="Ss/(Ss+Sr) i.e. SE")
        ax[1].plot(p_cuts[5:-5], bkg_eff_test[5:-5], color="C1", label="Bs/(Bs+Br)")
        ax[1].plot(p_cuts[5:-5], sig_pur_test[5:-5], color="deepskyblue", ls="--", label="Ss/(Ss+Bs) i.e. SP")
        ax[1].plot(
            p_cuts[5:-5], np.array(sig_eff_test[5:-5])*np.array(sig_pur_test[5:-5]), 
            color="darkblue", ls=":", lw=2, label="SE*SP"
        )
        ax[1].plot(
            p_cuts[5:-5], np.array(sig_cst_test[5:-5])/max(sig_cst_test), color="C2", label="Ss/(Ss+Bs)^0.5 (norm.)"
        )
        
        ax[0].axvline(opt_cut_train, lw=1, color="k", ls="--", label="optimal cut")
        ax[1].axvline(opt_cut_train, lw=1, color="k", ls="--", label="optimal cut")
        
        ax[0].legend()
        ax[0].grid()
        ax[1].grid()
        
        fig.supylabel("performance estimator")
        fig.supxlabel("cut on classifier output")
        fig.tight_layout()
        if bPerfPlots_save:
            fig.savefig("./output_misc/bdt_ab_cuttrends.png")

In [ ]:
if bBdtAb:  # set above
    
    # performance plots, after cut optimisation
    y_scores_train_int = (y_scores_train_float > output_cut).astype('float')
    y_scores_test_int = (y_scores_test_float > output_cut).astype('float')
    
    if True:  # (float) classifier output distributions
        bins = 100  # set nr. of bins for each distribution
        
        fig, ax = plt.subplots(figsize=(12, 5), ncols=2)
        fig.suptitle(
            "%d trees with max. depth %d, learning rate %d, min. tot. weight per leaf %.3f; output cut %f" \
            % (n_trees, max_depth, alpha, min_weight_fraction_leaf, output_cut)
        )
        fig.supxlabel("classifier output")
        ax[0].set_title("training")
        ax[1].set_title("testing")
        
        for i, i_class in enumerate(df["class"].unique()):
            ax[0].hist(
                y_scores_train_float[df["class"][y_train.index]==i_class], bins=bins,
                histtype="step", label="true "+[s for s in classlabel if classlabel[s]==i_class][0]
            )
            ax[1].hist(
                y_scores_test_float[df["class"][y_test.index]==i_class], bins=bins,
                histtype="step", label="true "+[s for s in classlabel if classlabel[s]==i_class][0]
            )
        ax[0].axvline(output_cut, ls="--", color="k", label="optimal cut")
        ax[1].axvline(output_cut, ls="--", color="k", label="optimal cut")
        
        ax[0].grid()
        ax[1].grid()
        ax[1].legend()
        
        fig.tight_layout()
        if bPerfPlots_save:
            fig.savefig("./output_misc/bdt_ab_outputopt.png")
    
    if True:  # confusion matrix
        fig, ax = plt.subplots(figsize=(12, 5), ncols=2)
        fig.suptitle(
            "%d trees with max. depth %d, learning rate %d, min. tot. weight per leaf %.3f; output cut %f" \
            % (n_trees, max_depth, alpha, min_weight_fraction_leaf, output_cut)
        )
        
        ConfusionMatrixDisplay(
            confusion_matrix=confusion_matrix(y_train, y_scores_train_int, normalize=confusionmatrix_norm),
            display_labels=[whichbkg, whichsig],
        ).plot(ax=ax[0])
        ConfusionMatrixDisplay(
            confusion_matrix=confusion_matrix(y_test, y_scores_test_int, normalize=confusionmatrix_norm),
            display_labels=[whichbkg, whichsig],
        ).plot(ax=ax[1])
        
        ax[0].set_title("training (%d events; norm. rule: %s)" % (y_train.shape[0], confusionmatrix_norm))
        ax[1].set_title("testing (%d events; norm. rule: %s)" % (y_test.shape[0], confusionmatrix_norm))
        
        fig.tight_layout()
        if bPerfPlots_save:
            fig.savefig("./output_misc/bdt_ab_confusionmatrixopt.png")

In [ ]:
# save Pickle file with classifier

# set notes to be saved in the model manually here:
notes = "manually selected parameters (which maximise accuracy)"

if bBdtAb & bClassifier_save:  # set above
    cls = {
        "classifier" : bdt_ab,
        "parameter_choice" : "manual" if bManualSet else "grid-borne",
        "output_cut" : {
            "training" : opt_cut_train,
            "testing" : opt_cut_test,
            "manual" : output_cut_man,
            "used_for_evaluation" : "manual" if bManualCut else ("training" if bAutoCutTrain else "testing")
        },
        "variables" : list(X.columns),
        "weight_rule" : "physical" if bReweightStat else "normalised",
        "weight_range_lambda" : limLambdaWeights,
        "signal" : {
            "class" : whichsig,
            "training_population" : y_train[y==1].shape[0],
            "testing_population" : y_test[y==1].shape[0],
        },
        "background" : {
            "class" : whichbkg,
            "training_population" : y_train[y==0].shape[0],
            "testing_population" : y_test[y==0].shape[0],
        },
        "notes" : notes,
    }, 
    pickle.dump(cls, open("./output_misc/bdt_ab.pickle", "wb"))
    print("output pickle file saved with note:")
    print(notes)